In [1]:
import os
import sys
import yaml

import tensorflow as tf
from tensorflow.keras.utils import Progbar
import tensorflow.keras as K
import librosa
import numpy as np
import glob

2024-04-28 11:26:10.971513: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64:/usr/local/lib:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64:
2024-04-28 11:26:10.971528: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

from model.dataset import Dataset
from model.fp.melspec.melspectrogram import get_melspec_layer
from model.fp.specaug_chain.specaug_chain import get_specaug_chain_layer
from model.fp.nnfp import get_fingerprinter


# Functions definition

In [37]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass==False) # Detachable by setting m_specaug.bypass.

    # m_fp: fingerprinter g(f(.)).
    m_fp = get_fingerprinter(cfg, trainable=False)
    return m_pre, m_specaug, m_fp

def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

@tf.function
def test_step(X, m_pre, m_fp):
    """ Test step used for mini-search-validation """
    X = tf.concat(X, axis=0)
    feat = m_pre(X)  # (nA+nP, F, T, 1)
    m_fp.trainable = False
    emb_f = m_fp.front_conv(feat)  # (BSZ, Dim)
    emb_f_postL2 = tf.math.l2_normalize(emb_f, axis=1)
    emb_gf = m_fp.div_enc(emb_f)
    emb_gf = tf.math.l2_normalize(emb_gf, axis=1)
    return emb_f, emb_f_postL2, emb_gf # f(.), L2(f(.)), L2(g(f(.))

@tf.function
def predict(X, m_pre, m_fp):
    """ 
    Test step used for mini-search-validation 
    X -> (B,1,8000)
    """
    feat = m_pre(X)  # (nA+nP, F, T, 1)
    m_fp.trainable = False
    emb_f = m_fp.front_conv(feat)  # (BSZ, Dim)
    emb_f_postL2 = tf.math.l2_normalize(emb_f, axis=1)
    emb_gf = m_fp.div_enc(emb_f)
    emb_gf = tf.math.l2_normalize(emb_gf, axis=1)
    return emb_f, emb_f_postL2, emb_gf # f(.), L2(f(.)), L2(g(f(.))

# Load latest checkpoint

In [4]:
checkpoint_name_dir:str = "./logs/CHECKPOINT_BSZ_120"#"CHECKPOINT"   # string
checkpoint_index:int = None  # int
config:str = "default"   

In [5]:
cfg = load_config(config)


cli: Configuration from ./config/default.yaml


In [6]:
m_pre, m_specaug, m_fp = build_fp(cfg)

2024-04-28 11:26:42.175676: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-04-28 11:26:42.186565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-28 11:26:42.186719: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.725GHz coreCount: 82 deviceMemorySize: 23.68GiB deviceMemoryBandwidth: 871.81GiB/s
2024-04-28 11:26:42.186794: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-28 11:26:42.186917: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0

In [7]:

checkpoint = tf.train.Checkpoint(m_fp)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_name_dir))

# Load data

In [12]:
audio_path = ""

In [ ]:
audio,fs = librosa.load(audio_path, mono=True, sr=22050)

In [8]:
dataset = Dataset(cfg)

In [21]:
test_db_ds = dataset.get_test_dummy_db_ds()

In [27]:
len(test_db_ds)

430034

In [28]:
X = test_db_ds.__getitem__(0)

In [42]:
X[0][0][None,:].shape

(1, 1, 8000)

In [30]:
Y = tf.concat(X, axis=0)

In [31]:
Y.shape

TensorShape([125, 1, 8000])

# Model Predict

In [43]:
emb_f, emb_f_postL2, emb_gf = predict(X[0][0][None,:], m_pre, m_fp)

In [45]:
emb_gf.shape

TensorShape([1, 128])

In [46]:
emb_gf

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 0.07539804, -0.16872996, -0.12115411, -0.01941191, -0.0148505 ,
         0.08560746, -0.0388802 ,  0.11214361,  0.02991246,  0.12607488,
         0.00300827,  0.06731218, -0.01309047,  0.03161237, -0.12762582,
         0.04633382,  0.11228829,  0.01818596, -0.09356156, -0.08653133,
        -0.15511537, -0.10948697, -0.00143263,  0.06219649,  0.00831405,
         0.04150734, -0.0522717 ,  0.0057195 , -0.0325827 , -0.03062948,
         0.03439471,  0.03571294, -0.08395135,  0.01864311,  0.03021016,
         0.02572118, -0.00137527,  0.01199023, -0.21145529, -0.03078636,
        -0.09409375,  0.08832733,  0.02622688, -0.0805587 , -0.04767458,
        -0.07374059, -0.07926158, -0.00388286,  0.06218813, -0.09946233,
        -0.14341326,  0.10287136,  0.0331257 ,  0.02296653,  0.05973741,
        -0.1981869 ,  0.04241665, -0.18951239,  0.12496328, -0.08082911,
        -0.00096449, -0.03653152, -0.02718567, -0.11009899,  0.1064317 ,
 

# System Audio Fingerprinting

In [50]:
files_db_dir = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-dummy-db-100k-full/fma_full'
files_query_dir = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-dummy-db-100k-full/fma_full'

In [47]:
d = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-query-db-500-30s/test_ids_icassp2021.npy'

In [51]:

y = np.load(d)

In [49]:
y

array([14655, 21662,  7594, ..., 15722,  1097,  4538])

In [52]:
files = glob.glob(os.path.join(files_db_dir,"**/*.wav"))

In [53]:
len(files)

93458